<h1> Veeam - Powershell Snapin - Jobs Configuration </h1>

In [ ]:
import pandas as pd
import re
import datetime as dt
import csv
import os
from zipfile import ZipFile as zipfile

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

# veeam info zip file path created from veeam_backup_job_info.ps1
zip_file = f'{export_folder}/CUWestVeeamReport.zip'

 ### Import Modules, Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# import and assign secrets from env.ini

# Get all job info csv

In [ ]:
## Unzip veeam_job_info.zip

with zipfile(zip_file, 'r') as z:
    z.extractall(export_folder)

In [ ]:
job_files = []

for root, dirs, files in (os.walk(f'{export_folder}/job_info/')):
    for file in files:
        file_path = root + file
        size = os.path.getsize(file_path)
        print(size)
        if size != 0:
            job_files.append(file_path)

for file in job_files:
    # try:
    df = pd.read_csv(file)
    if df_veeam_jobs.empty:
        pd.concat([df_veeam_jobs,df],ignore_index=True)
        print("concat")
    else:
        df_veeam_jobs = df
        print("df_veeam_jobs created")


    # except Exception as e:
    #     print(e)
    #     print(f'CSV:{file} is empty')

In [ ]:
df_veeam_jobs

In [ ]:
for file in job_files:
    try:
        df = pd.read_csv(file)
        if df_veeam_jobs:
            pd.concat([df_veeam_jobs,df],ignore_index=True)
            print("concat")
        else:
            df_veeam_jobs = df
            print("df_veeam_jobs created")
    except:
        print(f'CSV:{file} is empty')

In [ ]:
df_veeam_jobs

In [ ]:
df

In [ ]:
schedule_prog = re.compile(r'([^\:]+)' #Start time
                           r'\:\s\[' #: [
                           r'([\d\/\s\:PMA]+)' #11/4/2019 8:00:00 PM
                           r'\]\,\s'  #],
                           r'([^\:]+)' #Latest run time
                           r'\:\s\[' #: [
                           r'([\d\/\s\:PMA]+)' #11/7/2019 8:00:14 PM
                           r'\]\,\s' #],
                           r'([^\:]+)' #Timeout for backup completion
                           r'\:\s\[' #: [
                           r'([\dminhr]+)' #180min
                           r'\]\,\s' #],
                           r'([^\:]+)' #Next run time
                           r'\:\s\[' #: [
                           r'([\d\w\s]?)' #''
                           r'\]\,\s' #],
                           r'([^\:]+)' #Retry times on failure
                           r'\:\s\[' #: [
                           r'([\d\w\s]?)' #'6'
                           r'\]\,\s' #],
                           r'([^\:]+)' #Retry timeout
                           r'\:\s\[' #: [
                           r'([\d\w\s]+)' #20 min
                           r'\]\,\s' #],
                           r'([^\:]+)' #Daily options
                           r'\:\s\[' #: [
                           r'([^]]+)' #Enabled: True, ...
                           r'\]') #],

In [ ]:
df = df.dropna(how='all',axis=1)

In [ ]:
def start_time(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][1]

def last_run_time(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][3]

def timeout_for_backup_completion(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][5]

def next_run_time(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][7]

def retry_times_on_failure(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][9]

def retry_timeout(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][11]

def schedule_options_type(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    return result[0][12]

In [ ]:
df.iloc[:]['startTime'] = df['ScheduleOptions'].apply(start_time)
df.iloc[:]['lastRunTime'] = df['ScheduleOptions'].apply(last_run_time)
df.iloc[:]['jobTimeout'] = df['ScheduleOptions'].apply(timeout_for_backup_completion)
df.iloc[:]['jobRetryCount'] = df['ScheduleOptions'].apply(retry_times_on_failure)
df.iloc[:]['scheduleOptionsType'] = df['ScheduleOptions'].apply(schedule_options_type)

In [ ]:
df

In [ ]:
daily_options_prog = re.compile(r'([^\:]+)\:\s([^\,]+)\,\s([^:]+)\:\s([^\,]+)\,\s([^\:]+)\:\s([^\r\n]+)')
def backup_schedule_enabled(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    result_str = str(result[0][13])
    subresult = (daily_options_prog.findall(result_str))
    for end_result in subresult:
        return (end_result[1])

def backup_schedule_type(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    result_str = str(result[0][13])
    subresult = (daily_options_prog.findall(result_str))
    for end_result in subresult:
        return (end_result[3])

def backup_schedule_details(ScheduleOptions):
    result = schedule_prog.findall(ScheduleOptions)
    result_str = str(result[0][13])
    subresult = (daily_options_prog.findall(result_str))
    for end_result in subresult:
        return (end_result[5])

In [ ]:
df.iloc[:]['jobScheduleEnabled'] = df['ScheduleOptions'].apply(backup_schedule_enabled)
df.iloc[:]['jobScheduleType'] = df['ScheduleOptions'].apply(backup_schedule_type)
df.iloc[:]['jobScheduleDetails'] = df['ScheduleOptions'].apply(backup_schedule_details)

In [ ]:
veeam_job_report_cols = []
with open('d:/git/example_infrastructure_data_dev/dictionaries/veeam_job_report.dict','r') as f:
    reader = csv.reader(f)
    for row in reader:
        print((row[0]))
        veeam_job_report_cols.append(row[0])

In [ ]:
veeam_job_report_cols

In [ ]:
df_veeam_job_report = df[veeam_job_report_cols]

In [ ]:
df_veeam_job_report.to_csv(export_folder + 'veeam_job_report_' + current_time + '[REDACTED]/.csv')

In [ ]:
df = pd.read_csv('[REDACTED]/.csv')

In [ ]:
df.columns

In [ ]:
VssSnapshotOptions: [Enabled: [True], ApplicationProcessingEnabled: [True], IgnoreErrors: [False], IsCopyOnly: [False], UsePersistentGuestAgent: [False]], WinGuestFSIndexingOptions: [Type: [None]], LinGuestFSIndexingOptions: [Type: [None]], SqlBackupOptions: [TransactionLogsProcessing: [TruncateOnlyOnSuccessJob], BackupLogsFrequencyMin: [15], UseDbBackupRetention: [True], RetainDays: [15], ProxyAutoSelect: [True]FailJobOnDbAbsenceOrBackupImpossibility: [False]], ExchangeBackupOptions: [TransactionLogsProcessing: [TruncateOnlyOnSuccessJob]], WinCredsId: [[REDACTED_UUID]], LinCredsId: [00000000-0000-0000-0000-000000000000], Old Properties: [Enabled: [True], IgnoreErrors: [False], GuestFSIndexingType: [None], IncludedIndexingFolders: [], ExcludedIndexingFolders: [], LinGuestFSIndexingType: [None], LinIncludedIndexingFolders: [], LinExcludedIndexingFolders: [], IsFirstUsage: [True]]
